In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/home/kaushal/mystuff/thinklang/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

os.environ["HF_HOME"] = os.getcwd() + "/cache/models"

In [5]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16)

In [14]:
path = '/home/kaushal/mystuff/thinklang/examples'

def gather_initial_prompt():
    training_codes = ""
    sample_codes = [f"{path}/{f}" for f in os.listdir(path=path)]

    # map them into a single string and feed that into the model as context
    for code_loc in sample_codes:
        with open(code_loc) as f:
            training_codes += f.read()
            training_codes += "\n"

    return training_codes

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16)
messages=[
    { 'role': 'user', 'content': f"""You are a software developer writing code in a new language. The language syntax is shown below:
        context: {gather_initial_prompt()}

        You are to code in this langauge using this syntax. Write just the source code without any explanation.
        
        Prompt: Write a fizzbuzz program using the syntax specified in the context."""}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Here is a simple FizzBuzz program in the specified language:

```python
for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)
```

This program uses a for loop to iterate over the numbers 1 to 100. For each number, it checks if the number is divisible by 3 and 5, if so, it prints "FizzBuzz". If not, it checks if the number is divisible by 3, if so, it prints "Fizz", if the number is divisible by 5, it prints "Buzz", and if the number is not divisible by either, it prints the number itself.

